# Valutazione

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random

In [2]:
# lettura dei rating normalizzati dal relativo CSV
pd.set_option('display.max_columns', 500)
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')
# lettura dei rating non normalizzati dal relativo CSV
ratings_df = pd.read_csv('./data/jester_jokes_ratings.csv')

## Separazione dei rating normalizzati

Si separano i rating noramlizzati in due parti: 
1. una parte contenente tutti gli utenti che hanno valutato tutte e 100 le barzellette;
2. una parte contenente tutti gli utenti che non hanno valutato tutte le barzellette.

Si seleziona poi l'utente target (al quale si vuole fare una raccomandazione) dalla matrice sparsa e si cercano i vicini di quell'utente nella la matrice non sparsa.

In [3]:
# utenti che hanno valutato tutte e 100 le barzellette
complete_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] == 100]
print('totale utenti che hanno valutato tutte le barzellette: ', len(complete_ratings))

# utenti che non hanno valutato tutte le barzellette
sparse_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] != 100]
print('totale utenti che non hanno valutato tutte le barzellette: ', len(sparse_ratings))

totale utenti che hanno valutato tutte le barzellette:  7200
totale utenti che non hanno valutato tutte le barzellette:  17783


In [4]:
# l'idea è di creare un nuovo dataframe senza alcuni elementi random. 
# su questo dataframe possiamo fare predizioni e possiamo poi controllare
# se queste sono corrette andando a guardare quello completo
def create_training_set(training_ratings):
    # per ogni riga della matrice:
    for r in range(training_ratings.shape[0]): 
        # creo una lista random dove i numeri mi rappresentano le posizioni dei nan
        randomList = random.sample(range(1, 101), 20)
        # per ogni elemento della lista:
        for col in randomList: 
            # metto nan 
            training_ratings.iloc[r, training_ratings.columns.get_loc("joke_"+str(col))] = np.nan
            # aggiorno il numero di jokes complessivi
            training_ratings.iloc[r, training_ratings.columns.get_loc("number_of_jokes_rated")] = 80
    return training_ratings

training_ratings = complete_ratings.copy()        
create_training_set(training_ratings)
complete_ratings

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
1,2,100,1.3337,-3.0363,3.6137,1.6237,-5.1263,-12.4063,-3.4763,-8.0863,6.1337,6.4737,4.0037,5.8937,1.6737,4.6837,1.8137,-3.7163,1.9137,-3.4263,0.5537,-3.9563,-1.8763,5.8937,5.6037,6.4237,-2.6963,4.8237,1.9637,-1.8763,-3.1363,4.2437,3.7537,-3.6663,4.3937,6.2837,-4.5463,-2.0163,4.3437,0.6537,-3.6163,5.1637,5.0737,-6.5763,5.8937,6.2337,-7.0663,-5.9463,-7.6463,-3.6663,-4.2063,0.8937,0.4137,6.2837,-1.7763,-4.0563,-9.2463,-5.9463,5.8937,-4.8863,-2.6463,6.2837,-9.4463,-6.0963,-11.7763,1.7237,1.3337,-6.5763,5.9937,-1.6263,-1.9663,4.7737,-7.7463,0.0237,5.5537,5.0237,4.5837,3.4637,4.9737,6.2337,5.8937,5.4537,1.1837,2.1037,2.1037,3.3237,6.2337,1.7637,-2.7963,0.9437,1.8137,-2.1663,0.0737,-7.6963,-3.0363,5.1137,-2.9363,-4.8863,0.3137,-2.4063,-7.0663,-1.6763
5,6,100,-1.2723,1.3577,5.3377,-3.6023,-2.1923,0.5777,-3.7923,4.0277,-1.7523,3.0977,-1.9023,-0.8323,-0.1023,-3.6923,5.3877,-4.0323,1.2077,2.7177,2.6177,-1.2223,1.8877,4.3177,2.5177,-2.8723,2.6677,2.6177,-0.3423,-0.6323,4.2177,-2.0423,0.9677,4.5577,-1.7023,-0.5923,3.3477,4.4077,-1.3123,-3.1623,-0.4923,-2.1423,-3.9323,-3.0123,5.9677,3.0077,-1.4623,0.6777,-4.3723,-4.9123,-2.9223,4.2677,-1.4123,-2.5823,-1.1723,1.4477,1.6977,5.4277,-4.3723,-4.6623,-0.5923,3.7777,-1.7523,2.0377,0.2877,-4.5223,-3.0123,-4.9123,-4.5223,-4.0823,2.9577,-2.0923,-4.7623,-0.2923,-0.1023,0.4777,-2.3823,-1.6023,0.5777,-2.0423,-4.9523,5.6277,-1.3123,2.0377,-2.7223,-1.2223,4.6077,3.4877,0.9677,2.9577,-0.7823,0.1877,1.3577,-1.9923,4.2177,1.9377,2.7177,1.5477,4.9477,-4.1823,-0.1523,1.4477
7,8,100,9.2269,5.5469,11.5569,-3.8231,-5.7731,0.6869,11.6569,3.7969,-2.8031,-2.0331,10.5869,-5.4731,-4.5531,-5.5731,2.6769,-7.5131,-4.7031,-4.7931,3.4069,2.0969,-2.3231,-5.0431,3.5069,-5.8631,6.1769,2.4869,5.8369,-1.0631,9.9569,-1.7831,1.9469,11.6569,8.2169,-1.0131,-3.0531,3.9869,-4.7531,5.3969,-3.6831,8.0669,-4.0731,-2.0331,-6.5931,2.9169,1.1269,-2.0331,-6.5431,11.3169,2.2869,9.8169,-2.1231,-4.9931,6.5569,-6.6431,0.2469,3.6969,-3.2931,-6.6931,-3.8231,4.8669,-0.8131,-3.7331,-0.5231,-6.8331,-0.2331,-4.9431,-0.8631,-1.8331,-7.4231,0.2969,-7.5131,-1.4031,2.8769,-5.4731,-7.4631,4.1869,-1.7831,0.0069,-6.7431,-6.4931,5.9269,-2.1731,-3.0531,-3.5831,-7.5131,2.0469,-1.7431,2.2869,2.1469,-5.5731,9.6169,1.2669,2.2869,-3.2931,-0.7731,-0.9631,4.5269,2.3369,3.6969,2.3869
8,9,100,-2.9956,-2.7456,-8.6256,-6.0956,-7.9456,0.5044,-4.4956,-8.1356,-7.0656,-0.8056,-2.1156,0.5044,-4.0556,0.3044,-7.9456,-6.1956,-7.9456,-2.1156,-2.5556,0.5044,4.7744,-0.8056,0.5044,2.0044,4.5844,3.7044,5.6444,3.0744,1.7644,0.5044,7.1544,1.3244,0.5044,5.4544,4.1444,1.7644,-2.3156,-1.0456,5.2144,-3.6256,-0.6156,2.2044,-4.5456,-6.1956,2.2044,5.8944,2.6344,0.5044,1.3244,0.5044,-6.1956,-2.7456,7.1544,2.6344,4.1444,0.5044,0.5044,-4.4956,-6.0956,-4.4956,2.0044,6.3244,2.2044,-5.7056,5.6444,5.6444,3.4144,2.2044,7.1544,2.2044,-3.9656,4.9644,-1.2456,-5.5156,0.5044,0.5044,1.5744,0.5044,0.5044,0.5044,0.5044,-2.4056,0.5044,-2.4056,-2.6056,0.5044,0.5044,6.5244,5.3544,-1.4356,5.1644,0.5044,4.9644,0.5044,0.5044,0.5044,0.5044,0.5044,-2.6056,-4.1556
11,12,100,0.8691,1.3591,2.1291,-2.8209,0.2891,0.2891,-1.4109,4.5591,-7.6709,-1.8009,3.3891,1.3091,5.1891,-3.3009,-2.2409,-2.4809,5.0891,-0.7309,-1.

In [5]:
training_ratings

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
1,2,20,1.3337,-3.0363,3.6137,1.6237,NaN,-12.4063,-3.4763,-8.0863,NaN,NaN,4.0037,5.8937,1.6737,4.6837,1.8137,-3.7163,1.9137,-3.4263,0.5537,-3.9563,NaN,5.8937,5.6037,6.4237,-2.6963,4.8237,1.9637,NaN,-3.1363,4.2437,3.7537,-3.6663,4.3937,6.2837,-4.5463,-2.0163,4.3437,0.6537,-3.6163,5.1637,5.0737,-6.5763,5.8937,NaN,NaN,-5.9463,-7.6463,-3.6663,-4.2063,0.8937,0.4137,6.2837,NaN,-4.0563,NaN,-5.9463,5.8937,-4.8863,-2.6463,6.2837,-9.4463,-6.0963,-11.7763,1.7237,NaN,NaN,5.9937,NaN,-1.9663,4.7737,-7.7463,0.0237,NaN,5.0237,4.5837,3.4637,4.9737,6.2337,NaN,NaN,1.1837,2.1037,2.1037,NaN,6.2337,NaN,NaN,0.9437,NaN,-2.1663,0.0737,-7.6963,-3.0363,5.1137,-2.9363,-4.8863,0.3137,-2.4063,-7.0663,NaN
5,6,20,NaN,1.3577,5.3377,-3.6023,-2.1923,0.5777,-3.7923,4.0277,NaN,3.0977,-1.9023,-0.8323,-0.1023,-3.6923,5.3877,-4.0323,1.2077,2.7177,2.6177,-1.2223,1.8877,4.3177,2.5177,NaN,2.6677,NaN,-0.3423,-0.6323,4.2177,-2.0423,0.9677,NaN,NaN,-0.5923,3.3477,4.4077,-1.3123,-3.1623,NaN,NaN,NaN,-3.0123,NaN,3.0077,NaN,0.6777,-4.3723,NaN,-2.9223,4.2677,-1.4123,-2.5823,NaN,1.4477,1.6977,NaN,-4.3723,-4.6623,-0.5923,3.7777,NaN,2.0377,0.2877,-4.5223,NaN,-4.9123,-4.5223,-4.0823,2.9577,-2.0923,-4.7623,-0.2923,-0.1023,0.4777,-2.3823,NaN,0.5777,-2.0423,-4.9523,NaN,-1.3123,2.0377,-2.7223,-1.2223,4.6077,3.4877,0.9677,2.9577,NaN,0.1877,1.3577,-1.9923,NaN,1.9377,2.7177,1.5477,4.9477,-4.1823,-0.1523,1.4477
7,8,20,9.2269,NaN,11.5569,-3.8231,-5.7731,NaN,11.6569,3.7969,-2.8031,-2.0331,10.5869,-5.4731,-4.5531,-5.5731,2.6769,-7.5131,-4.7031,-4.7931,3.4069,2.0969,-2.3231,-5.0431,3.5069,-5.8631,6.1769,2.4869,5.8369,-1.0631,9.9569,-1.7831,1.9469,11.6569,NaN,-1.0131,NaN,3.9869,-4.7531,NaN,-3.6831,8.0669,-4.0731,NaN,-6.5931,NaN,1.1269,-2.0331,-6.5431,NaN,2.2869,9.8169,-2.1231,-4.9931,6.5569,-6.6431,0.2469,NaN,-3.2931,-6.6931,NaN,4.8669,-0.8131,-3.7331,-0.5231,-6.8331,-0.2331,-4.9431,NaN,-1.8331,-7.4231,NaN,-7.5131,-1.4031,NaN,NaN,-7.4631,4.1869,NaN,0.0069,NaN,-6.4931,5.9269,-2.1731,NaN,-3.5831,-7.5131,2.0469,-1.7431,2.2869,2.1469,NaN,9.6169,1.2669,2.2869,-3.2931,NaN,NaN,4.5269,2.3369,3.6969,2.3869
8,9,20,-2.9956,-2.7456,-8.6256,NaN,NaN,0.5044,NaN,NaN,-7.0656,-0.8056,NaN,0.5044,-4.0556,0.3044,-7.9456,-6.1956,NaN,-2.1156,NaN,0.5044,4.7744,-0.8056,NaN,2.0044,4.5844,3.7044,5.6444,3.0744,1.7644,0.5044,7.1544,1.3244,0.5044,5.4544,NaN,1.7644,-2.3156,-1.0456,5.2144,-3.6256,-0.6156,2.2044,-4.5456,-6.1956,2.2044,5.8944,2.6344,0.5044,1.3244,0.5044,-6.1956,-2.7456,NaN,2.6344,4.1444,0.5044,NaN,-4.4956,-6.0956,-4.4956,2.0044,6.3244,2.2044,NaN,5.6444,5.6444,NaN,2.2044,NaN,2.2044,-3.9656,4.9644,-1.2456,-5.5156,0.5044,0.5044,1.5744,0.5044,0.5044,0.5044,0.5044,-2.4056,0.5044,NaN,-2.6056,NaN,0.5044,6.5244,5.3544,NaN,NaN,0.5044,4.9644,NaN,0.5044,0.5044,0.5044,NaN,-2.6056,-4.1556
11,12,20,NaN,1.3591,2.1291,-2.8209,0.2891,NaN,NaN,4.5591,-7.6709,-1.8009,NaN,1.3091,5.1891,NaN,NaN,-2.4809,5.0891,-0.7309,-1.0209,0.9191,1.9891,0.1891,NaN,-4.8109,-2.0909,-1.8009,7.1291,NaN,5.5291,-3.9409,-0.2009,3.8291,-1.1209,-4.2309,7.1791,4.7091,1.9891,-0.3409,NaN,NaN,0.0891,1.8891,-2.7209,-5.5909,NaN,-1.7009,1.5991,-1.8509,1.9391,NaN,1.5991,-2.4309,-3.2109,NaN,1.3091,NaN,-7.3309,-5.7309,0.2391,-2.8209,-0.5909,NaN,1.0191,-2.5809,